In [2]:
import numpy as np
import pandas as pd
import cv2
import moviepy


In [3]:
def region_selection(image):
    mask = np.zeros_like(image)
    if len(image.shape)>2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    rows, cols = image.shape[:2]
    bottom_left = [cols*0.1, rows*0.95]
    top_left = [cols*0.4, rows*0.6]
    bottom_right = [cols*0.9, rows*0.95]
    top_right = [cols*0.6, rows*0.6]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image


In [4]:
def hough_transform(image):
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_len = 20
    max_line_gap = 500
    return cv2.HoughLinesP(image, rho, theta, threshold, minLineLength=min_line_len, maxLineGap= max_line_gap)

In [5]:
def average_slope_intercept(lines):
    left_lines = []
    left_weights = []
    right_lines = []
    right_weights = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1==x2:
                continue
            slope = (y2-y1) / (x2-x1)
            intercept = y1-(slope*x1)
            length = np.sqrt(((y2-y1)**2)+((x2-x1)**2))
            if slope <0:
                left_lines.append((slope,intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    left_lane= np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) >0 else None
    print(left_lane)
    right_lane= np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) >0 else None
    print(right_lane)



In [6]:
def pixel_points(y1, y2, line):
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))


In [7]:
def lane_lines(image, lines):
    if lines is not None:
        left_lane, right_lane = average_slope_intercept(lines)
        y1 = image.shape[0]
        y2 = y1*0.6
        left_line=pixel_points(y1, y2, left_lane)
        right_line =pixel_points(y1, y2, right_lane)
    else:
        left_line = ((0,0), (0,0))
        right_line = ((0,0), (0,0))
    return left_line, right_line

In [8]:
def draw_lane_lines(image, lines, color=[255,0,0], thickness=12):
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line, color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

In [9]:
def frame_processor(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel_size = 5
    blurred= cv2.GaussianBlur(grayscale, (kernel_size, kernel_size), 0)
    lowt = 50
    hight = 150
    edges = cv2.Canny(blurred, lowt, hight)
    region = region_selection(edges)
    hough = hough_transform(region)
    result = draw_lane_lines(image, lane_lines(image, hough))
    return result


In [10]:
def process_video(test_video, output_video):
    input_video = moviepy.VideoFileClip(test_video, audio=False)
    processed = input_video.image_transform(frame_processor)
    processed.write_videofile(output_video, audio=False)


In [11]:
process_video("test2.mp4", "video_output.mp4")

[  -1.79217404 1308.20631409]
[   0.97294616 -270.43060771]


TypeError: cannot unpack non-iterable NoneType object